In [186]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text 

In [206]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
#from pinecone.grpc import PineconeGRPC as Pinecone
#from langchain.vectorstores import PineconeVectorStore  
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from uuid import uuid4
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import getpass
import pylibmagic
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone

In [96]:
#  import nltk
#  nltk.download('punkt')

In [207]:
load_dotenv()
# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')


# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

## Embedding model setup

In [208]:
#os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                             #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x3759db6d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x3759ce8b0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Pinecone Setup

In [ ]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "vitafy-prod"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

## Load Documents

In [ ]:
# directory = 'data'

# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()
#     return docs

# docs = load_docs(directory)

In [210]:

directory = 'data'

def load_docs(directory, tenant_id):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenant_id  

    return docs

tenant_id = "tenant456"
docs = load_docs(directory, tenant_id)

## Generate uniqueId for Docs

In [ ]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

#uuids = [str(uuid4()) for _ in range(len(docs))]
#vectorstore = vectorstore.add_documents(documents=docs, ids=uuids)
vectorstore = vectorstore.add_documents(documents=docs)


In [211]:
chunk_size = 1000 
chunk_overlap = 20

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

## Initialize Vector store

In [ ]:
index_name = "vitafy-prod"

# vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
# #vectorstore = PineconeVectorStore(index=index, embedding=embeddings)
# #vectorstore = PineconeVectorStore(split_docs, embeddings, index=index)
#vectorstore = PineconeVectorStore.from_documents(split_docs, index_name=index_name,embedding=embeddings)

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [222]:
docs

[Document(metadata={'source': 'data/tenant456/_Billings Module ✓.docx', 'tenantId': 'tenant456'}, page_content="support@vitafyhealth.com | 1-866-4-Vitafy\n\nBillings Module\n\nSeptember, 2024\n\nOverview\n\nManaging client billing and service requests is a seamless process with our system. Clients can be billed through a subscription model with the option to access additional services or memberships, and members can request specific services as needed. The billings module allows you to handle transaction statuses and receipts and provides a clear view of upcoming and failed transactions through dedicated tabs.\n\n Within the settings module, you can conveniently manage custom service offerings.\n\nIntegrated Billings\n\nBill clients for memberships or one time transactions straight from within the portal. The portal gives you the option for multiple payment methods including credit/debit, bank transfer and cash. \n\nManage Transactions\n\nScheduled transactions can be put on hold or ca

## qa_chain

In [105]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={
#         "k": 1,
#             "filter" : {
#         'source': {'$eq': 'data/xyz/Cases Module ✓.docx'}  
#     },
#             }
# )

# llm = ChatOpenAI(
#     model="gpt-4o",
#     temperature=0.0,
# )

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever(),

# )
# qa.invoke("how to issue a refund? ")

## Template

In [106]:
#query1 = "how to make a new case?"


## With and Without Knowledge Base


In [107]:
# print("Chat with knowledge:")
# print(qa.invoke(query1).get("result"))
# #print("\nChat without knowledge:")
# print(llm.invoke(query1).content)

## Filter based on built-in metadata [Source]

In [108]:
docs

[Document(metadata={'source': 'data/abc/_Billings Module ✓.docx', 'text': "support@vitafyhealth.com | 1-866-4-Vitafy\n\nBillings Module\n\nSeptember, 2024\n\nOverview\n\nManaging client billing and service requests is a seamless process with our system. Clients can be billed through a subscription model with the option to access additional services or memberships, and members can request specific services as needed. The billings module allows you to handle transaction statuses and receipts and provides a clear view of upcoming and failed transactions through dedicated tabs.\n\n Within the settings module, you can conveniently manage custom service offerings.\n\nIntegrated Billings\n\nBill clients for memberships or one time transactions straight from within the portal. The portal gives you the option for multiple payment methods including credit/debit, bank transfer and cash. \n\nManage Transactions\n\nScheduled transactions can be put on hold or canceled from the upcoming tab using th

## Retriveal QA chain [Final RAG Chain]

In [221]:

retriever = vectorstore.as_retriever(
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': 'tenant123'}  
    },
            }
)

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.1,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

answer = qa("How to add a new case?")
#answer = qa.invoke("How to issue a refund? ")
print(answer)


{'query': 'How to add a new case?', 'result': 'To add a new case, follow these steps:\n\n1. Go to the Cases module from the left menu bar.\n2. Click on the Create button.\n3. If you are creating a case for an existing client, type in the name of the client.\n4. If you are creating a case for a new patient, use the Add Patient button.\n5. Click next.\n6. Add service (Optional).\n7. Click Save.\n\nRemember that only cases assigned to you will be shown by default under the cases module.'}


In [110]:
docs

[Document(metadata={'source': 'data/abc/_Billings Module ✓.docx', 'text': "support@vitafyhealth.com | 1-866-4-Vitafy\n\nBillings Module\n\nSeptember, 2024\n\nOverview\n\nManaging client billing and service requests is a seamless process with our system. Clients can be billed through a subscription model with the option to access additional services or memberships, and members can request specific services as needed. The billings module allows you to handle transaction statuses and receipts and provides a clear view of upcoming and failed transactions through dedicated tabs.\n\n Within the settings module, you can conveniently manage custom service offerings.\n\nIntegrated Billings\n\nBill clients for memberships or one time transactions straight from within the portal. The portal gives you the option for multiple payment methods including credit/debit, bank transfer and cash. \n\nManage Transactions\n\nScheduled transactions can be put on hold or canceled from the upcoming tab using th

In [141]:
from langchain.vectorstores import VectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

def setup_retriever(vectorstore: VectorStore, tenantId_path: str):
    """Set up the retriever with a similarity search type and a document filter."""
    return vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 1,
            "filter": {
                'tenantId': {'$eq': tenantId_path}
            }
        }
    )

def setup_llm(model_name: str = "gpt-4", temperature: float = 0.1):
    """Initialize the ChatOpenAI language model."""
    return ChatOpenAI(
        model=model_name,
        temperature=temperature,
    )

def setup_prompt():
    """Define the prompt template for question answering based on document content."""
    template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Maintain a professional and helpful tone as though you are providing customer service based on document information.
4. Answer normal conversational questions such as "Hi", "Hello", etc., with "Hello, How can I assist you?".

Question: {question}

Context: {context}

Answer:
"""
    return ChatPromptTemplate.from_template(template)

def docs2str(docs):
    """Convert document content to a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

def create_rag_chain(retriever, llm, prompt):
    """Build the Retrieval-Augmented Generation (RAG) chain."""
    return (
        {"context": retriever | docs2str, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

def get_answer(question: str, retriever, llm, prompt):
    """Generate an answer to the question using the RAG chain."""
    rag_chain = create_rag_chain(retriever, llm, prompt)
    response = rag_chain.invoke(question)
    return response

def main(question: str, vectorstore: VectorStore, tenantId_path: str):
    """Main function to handle the entire process."""
    retriever = setup_retriever(vectorstore, tenantId_path)
    llm = setup_llm()
    prompt = setup_prompt()
    
    response = get_answer(question, retriever, llm, prompt)
    print(f"Question: {question}")
    print(f"Answer: {response}")

# Example usage
# vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
tenantId_path = "xyz"
question = "How to issue a refund?"

main(question, vectorstore, tenantId_path)


Question: How to issue a refund?
Answer: The answer is not specifically mentioned in the provided documents.


## Describe Vector Database

In [ ]:
pc.describe_index("vitafy-prod")


{'deletion_protection': 'enabled',
 'dimension': 1536,
 'host': 'test-3-unx28qm.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'test-3',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

## Delete by embeddedId

In [113]:
for ids in index.list(prefix='95467427-93bb-43a9-9df2-9c345d6a821f'):
    print(ids)
    index.delete(ids=ids)

In [225]:
 # Create a dummy vector with 1536 dimensions
vector = [0.1] * 1536

# Perform the query on the Pinecone index
result = index.query(
    namespace="",
    vector=vector,
    include_values=False,
    filter={
        "tenantId": {"$eq": 'abc'}
    },
    top_k=1,
    include_metadata=True
)

In [226]:
result

{'matches': [{'id': 'b47cfdc7-5378-4267-92fb-83fc8028ed16',
              'metadata': {'filename': 'Billings Module.docx',
                           'source': '/Users/ujjwal/Desktop/NoveltyFiles/Vitafy-AI-CHAT/data/abc/Billings '
                                     'Module.docx',
                           'tenantId': 'abc',
                           'text': 'support@vitafyhealth.com | 1-866-4-Vitafy\n'
                                   '\n'
                                   'Billings Module\n'
                                   '\n'
                                   'September, 2024\n'
                                   '\n'
                                   'Overview\n'
                                   '\n'
                                   'Managing client billing and service '
                                   'requests is a seamless process with our '
                                   'system. Clients can be billed through a '
                                   'subscr